In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 200)

PUBLIC_CSV = r"C:\Users\Askker\Desktop\HW Python\survey_results_public.csv"
SCHEMA_CSV = r"C:\Users\Askker\Desktop\HW Python\survey_results_schema.csv"

df = pd.read_csv(PUBLIC_CSV, low_memory=False)
schema = pd.read_csv(SCHEMA_CSV, low_memory=False)

print("df shape:", df.shape)
print("schema shape:", schema.shape)

# qname из схемы и пересечение колонок
qname_col = next((c for c in schema.columns if c.lower() == "qname"), schema.columns[0])
qnames = schema[qname_col].dropna().astype(str).str.strip().unique().tolist()
cols_in_schema = sorted(set(qnames).intersection(df.columns))
print("пересечение колонок со схемой:", len(cols_in_schema))



df shape: (49123, 170)
schema shape: (139, 6)
пересечение колонок со схемой: 124


In [2]:
def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# подбираем имена колонок под конкретный год
COL_RESPONSE_ID = pick_col(df, ['ResponseId', 'ResponseID', 'Respondent'])
COL_WORK_EXP    = pick_col(df, ['WorkExp', 'YearsCodePro', 'YearsCode'])
COL_REMOTE      = pick_col(df, ['RemoteWork', 'WorkRemote', 'Remote'])
COL_LANG_USED   = pick_col(df, ['LanguageHaveWorkedWith'])  # multi-select
COL_COMP        = pick_col(df, ['ConvertedCompYearly', 'CompTotal', 'ConvertedComp'])
COL_EDU         = pick_col(df, ['EdLevel', 'Education', 'EducationLevel'])
COL_AGE         = pick_col(df, ['Age', 'AgeBracket'])
COL_INDUSTRY    = pick_col(df, ['Industry'])
COL_COUNTRY     = pick_col(df, ['Country'])

def to_numeric_workexp(series):
    if series is None:
        return None
    mapping = {'less than 1 year': 0.5, 'more than 50 years': 51.0}
    def parse_val(x):
        if pd.isna(x): return np.nan
        if isinstance(x, (int, float)): return float(x)
        s = str(x).strip().lower()
        if s in mapping: return mapping[s]
        try:
            return float(s.replace('+', '').replace(',', '').split()[0])
        except:
            return np.nan
    return series.map(parse_val)

def multi_select_contains(series, token):
    if series is None:
        return pd.Series([False] * len(df))
    tok = str(token).strip().lower()
    return (
        series.fillna('').astype(str).str.lower()
        .str.split(';')
        .apply(lambda lst: tok in [x.strip() for x in lst])
    )


print("COL_WORK_EXP  =", COL_WORK_EXP)
print("COL_REMOTE    =", COL_REMOTE)
print("COL_LANG_USED =", COL_LANG_USED)
print("COL_COMP      =", COL_COMP)
print("COL_EDU       =", COL_EDU)
print("COL_AGE       =", COL_AGE)
print("COL_INDUSTRY  =", COL_INDUSTRY)
print("COL_COUNTRY   =", COL_COUNTRY)



COL_WORK_EXP  = WorkExp
COL_REMOTE    = RemoteWork
COL_LANG_USED = LanguageHaveWorkedWith
COL_COMP      = ConvertedCompYearly
COL_EDU       = EdLevel
COL_AGE       = Age
COL_INDUSTRY  = Industry
COL_COUNTRY   = Country


## 1) Сколько респондентов прошли опрос?

In [3]:

q1_total = len(df)
print("1) Всего респондентов:", q1_total)
q1_total


1) Всего респондентов: 49123


49123

## 2) Сколько респондентов ответили на все вопросы из схемы (schema[qname])?

In [4]:

subset = df[cols_in_schema]
q2_all = subset.notna().all(axis=1).sum()
print("2) Ответили на все вопросы из схемы:", q2_all)
q2_all


2) Ответили на все вопросы из схемы: 0


0

## 3) Меры центральной тенденции по опыту (WorkExp)

In [5]:

workexp_num = to_numeric_workexp(df["WorkExp"])

stats = pd.Series({
    "count":  workexp_num.notna().sum(),
    "mean":   workexp_num.mean(),
    "median": workexp_num.median(),
    "mode":   workexp_num.mode().iloc[0] if not workexp_num.mode().empty else np.nan,
    "std":    workexp_num.std(),
    "min":    workexp_num.min(),
    "max":    workexp_num.max(),
})

# округляем до 2 знаков
stats = stats.round(2)

stats.to_dict()





{'count': 42844.0,
 'mean': 13.37,
 'median': 10.0,
 'mode': 10.0,
 'std': 10.8,
 'min': 1.0,
 'max': 100.0}

## 4) Сколько респондентов работает удалённо?

In [6]:

if COL_REMOTE:
    remote_count = (df[COL_REMOTE].astype(str).str.lower() == "remote").sum()
    print("4) Удалённо работают:", remote_count)
    remote_count
else:
    print("Колонка RemoteWork не найдена.")


4) Удалённо работают: 10924


## 5) Какой процент респондентов программирует на Python?

In [7]:

if COL_LANG_USED:
    pct_python = multi_select_contains(df[COL_LANG_USED], "python").mean() * 100
    print("5) Процент, кто использует Python:", round(pct_python, 2))
    round(pct_python, 2)
else:
    print("Колонка языков не найдена.")


5) Процент, кто использует Python: 37.43


## 6) Сколько респондентов учились программировать на онлайн-курсах?

In [8]:

learn_col = next((c for c in df.columns if 'learn' in c.lower()), None)
if learn_col:
    online_cnt = multi_select_contains(df[learn_col], "online courses").sum()
    print(f"6) Колонка обучения: {learn_col}")
    print("Количество, кто учился по онлайн-курсам:", online_cnt)
    online_cnt
else:
    print("Колонка с вариантами обучения не найдена.")


6) Колонка обучения: LearnCodeChoose
Количество, кто учился по онлайн-курсам: 0


## 7) Среди Python-разработчиков — средняя и медианная годовая компенсация по странам

In [9]:
from IPython.display import display

if COL_LANG_USED and COL_COMP and COL_COUNTRY:
    df_comp = df.copy()
    df_comp['_is_python'] = multi_select_contains(df_comp[COL_LANG_USED], 'python')
    comp = pd.to_numeric(df_comp[COL_COMP], errors='coerce')

    by_country = (
        df_comp[df_comp['_is_python']]
        .assign(_comp=comp)
        .dropna(subset=[COL_COUNTRY, '_comp'])
    )

    result_7 = (
        by_country.groupby(COL_COUNTRY)['_comp']
        .agg(mean='mean', median='median', n='count')
        .sort_values('n', ascending=False)
    )

   
    result_7_fmt = result_7.assign(
        mean=result_7['mean'].round(2),     
        median=result_7['median'].round(2)  
    )

    print("7) Компенсации Python по странам (mean/median только для валидных сумм):")
    display(result_7_fmt.head(20))  # топ-20 стран

else:
    print("7) Не найдены нужные колонки для агрегации по странам.")
    result_7 = pd.DataFrame()


7) Компенсации Python по странам (mean/median только для валидных сумм):


,mean,median,n
Country,,,
United States of America,173271.39,150000.0,3115
Germany,86308.32,81210.0,1187
United Kingdom of Great Britain and Northern Ireland,117726.66,95299.0,747
France,69672.23,60328.0,588
India,33205.42,17436.0,540
Canada,102966.33,87550.0,491
Netherlands,95538.10,81210.0,331
Brazil,57053.15,27306.0,319
Spain,69761.91,63808.0,316


## 8) Уровни образования у 5 респондентов с наибольшей компенсацией

In [10]:
from IPython.display import display


if COL_COMP and COL_EDU:
    top5 = (
        df.assign(_comp=pd.to_numeric(df[COL_COMP], errors='coerce'))
          .dropna(subset=['_comp'])
          .sort_values('_comp', ascending=False)
          .head(5)[['_comp', COL_EDU, 'Country']]
          .rename(columns={'_comp': 'Comp', COL_EDU: 'EdLevel'})
    )

   
    top5_fmt = top5.copy()
    top5_fmt['Comp'] = top5_fmt['Comp'].round(0).apply(lambda x: f"{x:,.0f}")

    print("8) ТОП-5 компенсаций и уровни образования:")
    display(top5_fmt)
else:
    print("8) Нет нужных колонок компенсации/образования.")




8) ТОП-5 компенсаций и уровни образования:


,Comp,EdLevel,Country
34267,"50,000,000","Associate degree (A.A., A.S., etc.)",United States of America
28700,"33,552,715","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Ukraine
43143,"18,387,548","Associate degree (A.A., A.S., etc.)",Poland
35353,"15,430,267","Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Iran, Islamic Republic of..."
45971,"13,921,760","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Netherlands


## Бонус 1) По каждой возрастной категории — % кто программирует на Python

In [11]:
if COL_AGE and COL_LANG_USED:   
    age_python = (             
        pd.DataFrame({
            'Age': df[COL_AGE],  # берём возраст респондентов
            '_is_python': multi_select_contains(df[COL_LANG_USED], 'python')  
            
        })
        .dropna(subset=['Age'])      
        .groupby('Age')['_is_python']  
        .mean() * 100                  
    ).sort_index()                     

   
    age_python_df = (
        age_python.round(2)   
        .reset_index()        
        .rename(columns={'_is_python': '% Python'})  
    )

    # выводим заголовок и таблицу
    print("Бонус 1) Доля Python-разработчиков по возрастным категориям:")
    display(age_python_df)

else:
    # если нужных колонок нет
    print("Бонус 1) Нет нужных колонок Age/Languages")


Бонус 1) Доля Python-разработчиков по возрастным категориям:


,Age,% Python
0,18-24 years old,39.97
1,25-34 years old,36.82
2,35-44 years old,36.61
3,45-54 years old,38.48
4,55-64 years old,37.17
5,65 years or older,31.32
6,Prefer not to say,30.95


## Бонус 2) Среди респондентов в 75-м перцентиле компенсации и работающих удалённо — самые распространённые индустрии

In [12]:
if COL_COMP and COL_REMOTE and COL_INDUSTRY:   
    comp_num = pd.to_numeric(df[COL_COMP], errors='coerce')  
   
    p75 = np.nanpercentile(comp_num.dropna(), 75)  
    
    mask = (comp_num >= p75) & (df[COL_REMOTE].astype(str).str.lower() == 'remote')  
    
    top_industries = (
        df.loc[mask, COL_INDUSTRY]    
          .dropna()                    
          .value_counts()             
          .to_frame('count')          
          .head(20)                   
    )

    print("Бонус 2) ТОП-20 индустрий среди remote с компенсацией в топ-25%:")
    display(top_industries)

else:
    print("Бонус 2) Нет нужных колонок Compensation/Remote/Industry")


Бонус 2) ТОП-20 индустрий среди remote с компенсацией в топ-25%:


,count
Industry,
Software Development,1184
Fintech,189
Healthcare,188
Other:,176
"Internet, Telecomm or Information Services",138
Banking/Financial Services,88
Government,78
Media & Advertising Services,75
Retail and Consumer Services,64
